# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution



----------------------
## 1.1 Analyzing the data

Before actually diving into the data cleaning, we needed to get the data, understand it, and then understand what problems it could have. That is what we are doing in this first subpart.

### Importing the data, setting up the file

We wanted to be able to clean the data efficiently without setting up the whole database and thus chose pandas to work with the CSV file.
The first steps are quite straight-forward: import pandas library, get the url, and then extract the data with pandas. To get a first glance of the file, we looked at the first 20 lines of the file

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv"
df = pd.read_csv(url)
df.head(20)

------------------------
The database include the name of the I _part_ that was sold, the name of the _company_ , _country_ and _city_ of the company or just the shipping city, then the _price_ of the transaction and the _date_. 

The first few lines indicate that the price is written in different currencies, which we might want to change when handling the data.

We can also see that the database is by default sorted by date.

### Specifying the problems in John's record keeping

Knowing that John had been messy keeping track of the sales transaction, we used different tools to identify the problems:

- The first thing to do is to describe the database, in order to know global information about the data:


In [ ]:
df.describe()

--------------------
We can have a lot of information from this command:

- The count of the different columns shows us that some of the data is missing. We will investigate that just after.
- The number of parts, companies, countries and cities seem rather limited, i.e. we can manually check them, whereas price is quasi-unique each time, meaning that for the same part, the price varies (probably because of quantities ordered, but as they are not specified we can not be sure). Same goes for date, as it includes the time aswell, each transaction has a unique date attached to it (exception made for a few cases that we will solve).

Let us investigate the missing data problem.
To understand to which extend it is problematic more easily, we can just sum the null values for each column:

In [ ]:
df.isnull().sum()

------------------------------
The columns missing the most data are countries and cities, which is a good thing because we should be able to find out the missing data using the company name and the city of the transaction to fill in the country, and do the same for the missing cities. 
Hopefully no row will miss both country and city so we can fill everything automatically. If that was to happen, we could still fill the information by identifying a company's main city and country, and supposing those were the ones concerned in the transaction. But it would require some manual work.

To be able to generate a database without too much trouble, we want every column to be of the same type and the same format. That is what we are checking now, by looking at different unique values for each column (except price and date):
Note that by doing this, we also check for typos and other mistakes for values that should be the same.

In [ ]:
Part_Names = df.part.value_counts().index
print(Part_Names)

----------------------
The part names all seem correct and without typos, and their length difference should not be a problem when creating the database.

Now with the company names:

In [ ]:
Company_Names = df.company.value_counts()
print(Company_Names)

---------------------------

It seems like there is a lot of typos here, we will see how to solve these in the next subpart.

Also, Gabtune and Tagtune have both low appearance and similar name, so they could be the same company with a typo that has been repeated. We will now check that:

In [ ]:
print(df.loc[df.company=='Tagtune'])

print('\n-----------------------------\n')

print(df.loc[df.company=='Gabtune'])

---------------------------
We note that Tagtune and Gabtune are located in different countries and cities, which convinced us that they are two different companies

Then with the country names:

In [ ]:
Country_Names = df.country.value_counts().index
print(Country_Names)

---------------------------
Here, we seem to that we have various problems:

Some countries have typos, some have abbreviations, and some are translated in Danish.

Finally, we check the city names:

In [ ]:
City_Names = df.city.value_counts().index
print(City_Names)

------------------------
The cities seem rather fine from this index, apart from the extra tab in Amadora's name.

We now look at problems with dates:

In [ ]:
df.sort_values(by="date").head(10)

--------------------
Some dates are in a different format and to miss the time of the transactions. We will not be able to know the time, but we still might fix the format.

We might want to change the date format later to have an easier time creating the database.
We are now checking if changing the date format causes error:

In [ ]:
Date_Format = []
Date_Format = pd.to_datetime(df['date'], errors = 'coerce')
Date_Format = Date_Format.sort_values()
print(Date_Format.head(10))
print(Date_Format.tail(10))

--------------------
Some dates that we did not think were problematic actually turn into the NaT (Not a Time) when the format is changed. We will try to solve that in the next subpart.

## 1.2 Cleaning the Data

Now, we are actually trying to solve the problems.


First step is filling the missing country names, using the known city names and the country they lie in:

### Filling the missing values:

In order to do that, we use a mapping that fills in every missing country according to its corresponding city:


In [ ]:
map_base_country = df.dropna(subset=['country']).drop_duplicates('city').set_index('city')['country']
df['country'] = df['country'].fillna(df['city'].map(map_base_country))

df.isnull().sum()

------------------------
By showing the sum of null values again, we see that every missing country was filled, i.e. no row had a missing country and city.

Now we do the same for the city column this time, using the company as an index (some cities are in the same countries so we can not use countries as an index):

In [ ]:
map_base_city = df.dropna(subset=['city']).drop_duplicates('company').set_index('company')['city']
df['city'] = df['city'].fillna(df['company'].map(map_base_city))

df.isnull().sum()

-------------------------
Dropping the 10 missing parts and the price missing data as we do not have a way to identify them properly. Them representing a very little part of the global data and not being useable for business insights conforted us to simply delete those rows:

In [ ]:
df = df.dropna(how='any')

print(df.isnull().sum())
df = df.reset_index(drop=True)

-----------------------------
We finally have a table without any missing values now!

### Correcting typos

Now that companies, countries and cities are all complete, we can procede to correcting the typos:

In the company names:

In [ ]:
Typos_Companies = {'Zooxo.' : 'Zooxo', 'Laj0' : 'Lajo', ' -' : None, 'Ntagz' : 'Ntags', ' a' : None, 'aa' : None, 'Thoughtmixz' : 'Thoughtmix'}
df = df.replace(Typos_Companies)

df.company.unique()

--------------------
And in the the countries (adding in the city Amadora as well so we don't have to recall the method replace)

In [ ]:
#Replacing typos in the countries and in cities (Amadora)
Typos_Countries = {'Portuga' : 'Portugal', 'Tyskland' : 'Germany', 'US' : 'United States', 'Amadora\t' : 'Amadora'}
df = df.replace(Typos_Countries)


df.country.unique()

-------------------
Changing the unknown companies' names into None only made us able to solve the problem, which we do now with another mapping:

In [ ]:
map_base_companies = df.dropna(subset=['company']).drop_duplicates('city').set_index('city')['company']
df['company'] = df['company'].fillna(df['city'].map(map_base_companies))

df.isnull().sum()

-----------------------
As we saw, a set of dates had a problem with format. We will now solve those problems.

### Correcting problems with date formats

Some of them simply had a bad format, and did not indicate time. We will just set up an arbitrary time in order to have the format fit the other dates':

In [ ]:
df = df.replace(to_replace = '10/04/2017', value = '2017-04-10 00:00:00')
df.loc[df['date']=='2017-04-10 00:00:00']

---------------
While identifying the dates problems, we saw that transactions 3539 and 3540 have format problems, let us investigate that:

In [ ]:
print(df.iloc[3539])
print(df.iloc[3540])

---------------------
The problem seems to be that the indicated day is the 32nd, which of course does not exist.
We can check which day it was supposed to be by looking at previous and past operations:

In [ ]:
print(df.iloc[3538])
print(df.iloc[3541])

-----------------
And then correct it:

In [ ]:
Correct_Dates = {'2016-06-32 07:22:28' : '2016-06-10 07:22:28', '2016-06-32 08:08:48' : '2016-06-10 08:08:48'}
df = df.replace(Correct_Dates)
print(df.iloc[3539])
print(df.iloc[3540])

### Investigating companies with multiple cities:

We are now trying to see if the cities and the companies actually match 1 to 1, or if many companies are in the same cities, or if a company is based in different cities.

In [ ]:
Company_Group = df.groupby('company')
Company_City = Company_Group.apply(lambda x: x['city'].value_counts())
print(Company_City)

--------------------
As the orders in Monção for Brainsphere and the order in Vila Fria for Thoughtmix represent only a small part of the total orders for the companies, we decided to change them to the main city to reduce unnecessary cities: 

In [ ]:
Main_Cities =  {'Monção' : 'Braga', 'Vila Fria' : 'Amadora'}
df = df.replace(Main_Cities)

Company_Group = df.groupby('company')
Company_City = Company_Group.apply(lambda x: x['city'].value_counts())
print(Company_City)

## 1.3 Price reorganization

---

In order to make the next parts easier, we are trying to have a separate row for the currency and for the actual amount of the transaction. To do that, we define a splitString function that takes the price and splits it into 2 strings, one containing the amount, and the other one the currency.

In [ ]:
def splitString(str): 
  
    num = "" 

    special = "" 
    for i in range(len(str)): 
        if (str[i].isdigit() or str[i]=='.' or str[i]=='-' ): 
            num += str[i] 
        else: 
            special += str[i] 
    if special=='':
        special='€'
    return num, special

df1 = df.copy()
df1 = df1.reset_index(drop=True)
listprices = df1['price'].tolist()
listcurr=[]
listsymb=[]
for i in range(len(listprices)):
    a,b=splitString(listprices[i])
    listcurr.append(a)
    listsymb.append(b)

df1['amount']=pd.DataFrame(listcurr)
df1['currency']=pd.DataFrame(listsymb)

df1.currency.unique()

------------------------
To avoid having problems we did not see in the later parts, we quickly check that everything seems to be working just fine:

In [ ]:
df1.sample(20)

In [ ]:
# For readability purpose

currencydic={'€':'EUR', '£':'GBP', '$':'USD', '¥':'JPY'}
df1 = df1.replace(currencydic)

# Just to check
df1.tail(20)

----------------------
Now that we have separated the currency and the value, we check if it caused any problems :

In [ ]:
df1.loc[df1['amount'] == '']

In [ ]:
df1.loc[df1['amount'] == '-']

---------------
Now identifying why they were wrong in the first place:

*Note that we use df, the unmodified version, to check if our modification caused the problem or not*

In [ ]:
Problematic_Rows = [8934, 8683, 8684, 8685, 10328, 10329, 10330, 11154, 11155, 11156]
df.iloc[Problematic_Rows]

---------------
As we want data on prices, we remove the lines without price:

In [ ]:
df1 = df1.drop(Problematic_Rows, axis = 0)
df1 = df1.reset_index(drop=True)

---------------
And finally, we check if there are still problematic rows:

In [ ]:
df1.loc[df1['amount'] == '']

In [ ]:
df1.loc[df1['amount'] == '-']

### Global Data Check

Now that we have solved every problem we found, we can check again that everything seems fine:

In [ ]:
df1.sample(20)

In [ ]:
df1.isnull().sum()

## 1.4 Conclusion

---

After analyzing our initial data, cleaning it and then rearranging some parts, we obtained a dataframe that is ready to be enriched and then used. We have filled it by deducing the missing data we could, by dropping some unusable data and finally by changing the way it was presented.

## Part 2: Enriching the data

---

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

## 2.1 Importing and readjusting external data

---

Note that the prices are in differents currencies and the most common one is euro, so to make the values comparables, we convert all the currencies to Euro. In order to achieve this, we connect to an API to get data for currency rates at the specific point in time in which the transaction was made.

We start by importig the packages that we will use to request the data and to create a .csv file with the rates extracted

In [ ]:
import requests
import csv

-------------------
We set the date range to use:

In [ ]:
df1.sort_values(by="date")
start_date = df1.values[1][-3][0:10]
end_date = df1.values[-1][-3][0:10]
print("start_date = "+str(start_date))
print("end_date = "+str(end_date))

----------------
Now, it is time to request the data to the api and then save it into a .csv file:

In [ ]:
url_rates = "https://api.exchangeratesapi.io/history?start_at={}&end_at={}&symbols=USD,GBP,JPY".format(start_date,end_date)
r = requests.get(url_rates)
json_response = r.json()
rates = json_response['rates']

In [ ]:
file = open('rates.csv','w+')
csv_file = csv.writer(file)
file.write("Date,USD_rate,GBP_rate,JPY_rate\n")
for rate in rates:
    values = rates.get(rate)
    usd = values.get('USD')
    gbp = values.get('GBP')
    jpy = values.get('JPY')
    row = rate + "," + str(usd) + "," + str(gbp) + "," + str(jpy) + "\n"
    file.write(row)
file.close()

--------------------
Then, we use pandas to get a dataframe where dates are the indices and the columns represent the rates for USD, GBP and JPY:

In [ ]:
rates = pd.read_csv("rates.csv",encoding='utf-8-sig')
rates['Date'] = pd.to_datetime(rates.Date)
rates = rates.sort_values(by='Date')
rates = rates.set_index(['Date'])
rates.head(5)

## 2.2 Inserting the rates into our dataframe

----------------------

Finally, we make a match between the dataframe cointaining the rates and the dataframe with the prices in different currencies.

Since the exchange rates do not always match the transactions dates, we calculate the mean for the different currencies and use a "try/except" command to solve this issue, so that when the dates match, we use that day exchange rate, else, we try with the closest date up to three days back and if that does not work, we use the final except with the mean for the given currency exchange rate. It is also implemented a command to count the number of times that the calculation is made with the mean values of the currency and we get that less than one percent of the values were calculated with the average values.

In [ ]:
import datetime
df1["price_EUR"] = ""
e1, e2, e3= 0,0,0

rates_mean = rates.mean(axis = 0, skipna = True)
for trans in df1.iterrows():
    if (trans[1]['currency'] == 'EUR'):
        trans[1]['price_EUR'] = float(trans[1]['amount'])
    else:
        date = pd.to_datetime(trans[1]['date'][0:10])
        if (trans[1]['currency'] == 'USD'):
            try:
                rate = rates.loc[str(date)]['USD_rate']
            except KeyError:
                try:
                    rate = rates.loc[str(date+datetime.timedelta(days=-1))]['USD_rate']
                except KeyError:
                    try:
                        rate = rates.loc[str(date+datetime.timedelta(days=-2))]['USD_rate']
                    except KeyError:
                        try:
                            rate = rates.loc[str(date+datetime.timedelta(days=-3))]['USD_rate']
                        except KeyError:
                            rate = rates_mean[0]
                            e1+=1
            trans[1]['price_EUR'] = (float(trans[1]['amount']) / float(rate))
        elif (trans[1]['currency'] == 'GBP'):
            try:
                rate = rates.loc[str(date)]['GBP_rate']
            except KeyError:
                try:
                    rate = rates.loc[str(date+datetime.timedelta(days=-1))]['GBP_rate']
                except KeyError:
                    try:
                        rate = rates.loc[str(date+datetime.timedelta(days=-2))]['GBP_rate']
                    except KeyError:
                        try:
                            rate = rates.loc[str(date+datetime.timedelta(days=-3))]['GBP_rate']
                        except KeyError:
                            rate = rates_mean[1]
                            e2+=1
            trans[1]['price_EUR'] = (float(trans[1]['amount']) / float(rate))
        elif (trans[1]['currency'] == 'JPY'):
            try:
                rate = rates.loc[str(date)]['JPY_rate']
            except KeyError:
                try:
                    rate = rates.loc[str(date+datetime.timedelta(days=-1))]['JPY_rate']
                except KeyError:
                    try:
                        rate = rates.loc[str(date+datetime.timedelta(days=-2))]['JPY_rate']
                    except KeyError:
                        rate = rates_mean[2]
                        e3+=1
            trans[1]['price_EUR'] = (float(trans[1]['amount']) / float(rate))

In [ ]:
USD_p=e1/df1.currency.value_counts()[2]
GBP_p=e2/df1.currency.value_counts()[1]
JPY_p=e3/df1.currency.value_counts()[3]
print('Portion of the time that rates were converted using the mean value: %f GBP, %f USD, %f JPY'% (GBP_p,USD_p, JPY_p) )

df1.sample(20)

## 2.3 Conclusion

----------------------------

We now have a dataframe that has been enriched with the value of each transactions in euro instead of the original currency. We will now use this dataframe to create a database that is easier to use and that corresponds to what we want to use it for (here, optimize business operations).

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

## 3.1 Setting up Pandas dataframe to facilitate the database creation

---

Since there are redundancies in the dataframe that we are working on, we decided to create two tables, one that stores the companies attributes such as _id, company name, country_ and _city_  that we call **companies**, and another one, that contains all the transactions and is called **transactions**, which contains the attributes _part, company_id_ (foreign key from compamnies table), _price_ and _date_ 

We start by creating the table **companies** in pandas, and then write the INSERT command using a for loop.

In [ ]:
#dates to datetime and then create the database
df1['date']=pd.to_datetime(df1['date'], errors='coerce')

Unique_Companies = df1.groupby(['company','city']).apply(lambda df: df.sample(1))
Companies_df = Unique_Companies[['company', 'country', 'city']]
Companies_df['id'] = range(1, len(Companies_df) + 1)
Companies_df


In [ ]:
# Creating the Transactionsext Tables in Pandas
Transactions = df1.drop(df1.columns[[2, 4,6, 7]], 1)
Transactions.head(8)

## 3.2 Creating the SQL database

---

### Setting it up

---

To set up SQL our dataframe in the notebook, we firstly import SQLite3, then create a connection to the file, to finally put a cursor on the connection.

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('project1.db')

c = conn.cursor()

### Creating the companies table

---

For the table **companies**, we want to have an _id_ which will then be used as a foreign key in the **transactions** table.

In [ ]:
c.execute('CREATE TABLE IF NOT EXISTS companies ( id INTEGER PRIMARY KEY, company TEXT, country TEXT, city TEXT)')

conn.commit()

In [ ]:

string='INSERT INTO companies (company, country, city) VALUES '

a=[]
for i in range(len(Companies_df)):
    if i == (len(Companies_df)-1):
        a.append('("'+str(Companies_df.iloc[i,0])+'", "'+str(Companies_df.iloc[i,1])+'", "'+str(Companies_df.iloc[i,2])+'")')
        break 
    a.append('("'+str(Companies_df.iloc[i,0])+'", "'+str(Companies_df.iloc[i,1])+'", "'+str(Companies_df.iloc[i,2])+'"),')

b=string+"".join(a)

c.execute(b)


---

We check that the table is created and its values are correct.

In [ ]:
c.execute("select * from companies")

print(c.fetchmany(5))


### Creating a temporary table : transactionsext

---

The temporary table **transactionsext** and the table **companies** will be used to create **transactions**, the table that we will use in the queries. Finally, the table **transactionsext** can be removed.

In [ ]:
c.execute('CREATE TABLE IF NOT EXISTS transactionsext ( id INTEGER PRIMARY KEY, part TEXT, company TEXT, city TEXT, price INTEGER, date DATETIME)')

conn.commit()

In [ ]:
string1='INSERT INTO transactionsext (part, company, city, price, date) VALUES '

x=[]
for i in range(len(Transactions)):
    if i == (len(Transactions)-1):
        x.append('("'+str(Transactions.iloc[i,0])+'", "'+str(Transactions.iloc[i,1])+'", "'+str(Transactions.iloc[i,2])+'", '+str(Transactions.iloc[i,4])+', \''+Transactions.iloc[i,3].strftime("%Y-%m-%d %H:%M:%S")+'\')')
        break 
    x.append('("'+str(Transactions.iloc[i,0])+'", "'+str(Transactions.iloc[i,1])+'", "'+str(Transactions.iloc[i,2])+'", '+str(Transactions.iloc[i,4])+', \''+Transactions.iloc[i,3].strftime("%Y-%m-%d %H:%M:%S")+'\'),')
d =string1+"".join(x)

c.execute(d)


In [ ]:
c.execute("select * from transactionsext")

print(c.fetchmany(5))

### Creating the last table : transactions

---

The table is now set, we just need to create the final **transactions** table by using a subquery to match the _company_ and _city_ names from **transactionsext** to the _company_id_ in **companies**:

In [ ]:
c.execute('CREATE TABLE IF NOT EXISTS transactions AS SELECT companies.id AS \'company_id\', transactionsext.id AS \'transaction_id\', transactionsext.part, transactionsext.price,transactionsext.date FROM companies, transactionsext WHERE companies.company = transactionsext.company AND companies.city = transactionsext.city')
conn.commit()

c.execute("select * from transactions")

print(c.fetchmany(5))

In [ ]:
c.execute('DROP TABLE transactionsext;')
conn.commit()

## 3.3 Conclusion

---

In this third part, we have successfully created a SQL database using two main tables : **companies** and **transactions**. The first one matches, for each company, its name, its id, its city and the associated country.
The latter is intended to include each transactions, while being smaller than the original csv file as we removed the company, city and country column to only have the company id. The price of the transactions is also written in euro, so the conversion does not have to be done _à posteriori_.

The creation of this **transactions** table is made through the use of an extra table called **transactionsext**, which is an intermediate table that still has the _country_ and the _city_ instead of the _country_id_.

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---

### Importing additional libraries

---

Wanting to have an understandable way to explain the data to any data neophyte, we chose to graph our results using **matplotlib**. In order to ease the plotting, we also use **numpy** to shape our data in a more plottable way.

Lastly, we are using the **time** library for easier time handling.

In [ ]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import time

## 4.1 Finding the most devoted clients

---

The next plot shows who are the 5 most profitable clients (since the beginning of the registry):

In [ ]:
# Executing the SQLite query
query = "SELECT com.company, sum(trans.price) FROM transactions trans JOIN companies com ON com.id = trans.company_id GROUP BY trans.company_id ORDER BY sum(trans.price) DESC LIMIT 5"
c.execute(query)
result = c.fetchall()

# Initializing variables
companies = []
profits = []

# Storing the query results
for i in result:
    companies.append(i[0])
    profits.append(i[1])

# Plotting
plt.figure(figsize=(10, 5))
plt.bar(range(1,len(profits)+1), profits, 
        tick_label=companies, 
        color='forestgreen')
plt.xlabel("Companies")
plt.ylabel("Revenue (€)")
plt.title("Top 5 clients by revenue")
plt.show()

We see that Thoughtmix and Twitterbeat are dominating the other clients, followed by three very close customers : Zooxo, Shufflebeat and Cchatterbridge.

## 4.2 Finding clients whose revenues are declining

---

To detect the clients for which profit is declining, it is important to see the evolution of the incomes from all companies during the last complete year.

The clients have been separated in 4 groups following alphabetical order, each one with 7 clients, for readability purpose:

In [ ]:
# Initializing variables
incomes = {}
timestamps = []

# Getting and storing the available months from the DB
query = "SELECT DISTINCT(substr(date, 6, 2)) FROM transactions WHERE substr(date, 0, 5) = '2018' ORDER BY DATETIME(date) ASC;"
c.execute(query)
months = [elt[0] for elt in c.fetchall()]

# Getting and storing the available years from the DB
#query = "SELECT DISTINCT(substr(date, 0, 5)) FROM transactions ORDER BY DATETIME(date) ASC;"
#c.execute(query)
#years = [elt[0] for elt in c.fetchall()]

# Analyzing data from last year with data (2018)
years = ['2018']

# Getting and storing the company names from the DB
query = "SELECT DISTINCT(company) FROM companies"
c.execute(query)
companies = c.fetchall()
for company in companies:
    incomes[company[0]] = list()

# Getting and storing the incomes for every month
query = "SELECT com.company, sum(trans.price) FROM transactions trans JOIN companies com ON com.id = trans.company_id WHERE substr(trans.date, 0, 5) = '{}' AND substr(trans.date, 6, 2) = '{}' GROUP BY trans.company_id;"
for year in years:
    for month in months:
        timestamps.append(month+"-"+year)
        for key in incomes.keys():
            incomes[key].append(0)
        c.execute(query.format(year,month))
        result = c.fetchall()
        if len(result)>0:
            for com in result:
                del incomes[com[0]][-1]
                incomes[com[0]].append(com[1])

---

After having done the queries and stocked the data, we simply need to plot it. The next four graphs contain all the clients and its profits during the year 2018 (the last one with useful data):

In [ ]:
# Plotting
x = np.arange(len(months)*len(years))
count = 0
for i in range(1,5):
    plt.figure(figsize=(15, 5))
    legend = []
    for j in range(0,7):
        plt.plot(x, incomes[companies[count][0]])
        legend.append(companies[count][0])
        count += 1
    plt.legend(legend, loc='best')
    plt.title("Profits by client")
    plt.ylabel("Profits (€)")
    plt.xticks(x, timestamps, rotation='vertical')
    plt.show()

---

The income coming from each company seems rather variable over the course of the months, but it appears that Eimbee, Lajo, Teklist and Zooxo have been decreasing in the past months. Looking more closely at these companies could be a good idea to increase profits.

## 4.3 Finding the most profitable countries

---

If we know which country is the most profitable, we could have more specific advertisment campaigns, or just have more work done into translation in order to increase loyalty of customers from a given country. Also, it could help in the planning of creating affiliated companies that specially target a specific country.

The next plot shows the top 5 profitable countries (since the beggining of the registry):

In [ ]:
# Executing the SQLite query
query = "SELECT com.country, sum(trans.price) FROM transactions trans JOIN companies com ON com.id = trans.company_id  GROUP BY com.country ORDER BY sum(trans.price) DESC LIMIT 5;"
c.execute(query)
result = c.fetchall()

# Storing the results
countries = []
profits = []
for i in result:
    countries.append(i[0])
    profits.append(i[1])

# Plotting
plt.bar(range(1,len(profits)+1), profits, 
        tick_label=countries, 
        color='forestgreen')
plt.xlabel("Countries")
plt.ylabel("Profit (€)")
plt.title("Top 5 countries by profit")
plt.show()

## 4.4 Finding out which parts are sold the most

---

Another axis of optimizing business operations could be done on specific part, thus finding out which parts aquire us the most revenue could be helpfull to decide how to improve marketing, production or basically any topic where the part we sell has influence. 

In [ ]:
# Executing the SQLite query
query = "SELECT part, sum(price) FROM transactions GROUP BY part ORDER BY sum(price) DESC LIMIT 5;"
c.execute(query)
result = c.fetchall()

# Storing the results
parts = []
profits = []
for i in result:
    parts.append(i[0])
    profits.append(i[1])

# Plotting
plt.bar(range(1,len(profits)+1), profits, 
        tick_label=parts, 
        color=('r', 'b', 'g', 'y', 'purple'))
plt.xlabel("Products")
plt.ylabel("Revenue (€)")
plt.title("Top 5 products by revenue")
plt.show()

## 4.5 Conclusion

---

After answering a few different questions, we are offered plenty of usefull insights to give to our company, whether it is about companies, parts, countries or variability in time. Now that we know which country, which part, and which companies are the most profitable, the compagny may focus it's work on topics that will have an impact on the company's revenue, which is what we were hired for.

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

## 5.1 Current performance

---

With the current dataset the solution offers a good performance while retrieving useful information, but it is necessary to check if the behavior is going to be the same with a bigger dataset.

First, we are going to execute the queries using the current implementation of the system and measure the time of the performance.

In [ ]:
# Building the SQL query
company = "Thoughtmix"
start_date = "2017-01-01 00:00:00"
end_date = "2019-01-01 00:00:00"
query_1 = "SELECT * FROM transactions trans JOIN companies com ON com.id = trans.company_id WHERE com.company = '{}' AND trans.date BETWEEN '{}' AND '{}' ORDER BY trans.date ASC;"

# Executing the SQL query
start_time = time.time()
c.execute(query_1.format(company, start_date, end_date))
end_time = time.time()
test1 = end_time - start_time

# Printing results
print("Getting all sales to company " + company + " between time " + start_date + " and "
      + end_date + " (" + str(len(c.fetchall())) + " rows) in " + str(test1) + "secs")

In [ ]:
# Building the SQL query
rows = 100
query_2 = "SELECT * FROM transactions ORDER BY date DESC LIMIT {};"

# Executing the SQL query
start_time = time.time()
c.execute(query_2.format(rows))
end_time = time.time()
test2 = end_time - start_time

# Printing results
print("Getting the latest " + str(rows) + " sales in " + str(test2) + "secs")

In [ ]:
# Building the SQL query
query_3 = "SELECT SUM(price) AS total, date FROM transactions GROUP BY DATE(date);"

# Executing the SQL query
start_time = time.time()
c.execute(query_3.format(rows))
end_time = time.time()
test3 = end_time - start_time

# Printing results
print("Getting total sales per company per day (" + str(len(c.fetchall())) + " rows) in " 
      + str(test3) + "secs")

As we can see, the queries are executed in few milliseconds even when they are returning more than 1000 rows of data. But this time can be improved adding an index to the column that is being used to filter (the 'WHERE' clause in the SQL queries).

## 5.2 Creating indexes

---

In this case we are going to create an index for the _date_ column within the **transactions** table because we are filtering by time.

In [ ]:
# Reseting the connection to the database
c.close()
conn = sqlite3.connect('project1.db')
c = conn.cursor()

# Creating index in DATE column (table TRANSACTIONS)
index = "CREATE INDEX index_date ON transactions (date);"
c.execute(index)

After add an index to the _date_ column let's check the performance for the two first queries.

In [ ]:
# Executing the SQL query
start_time = time.time()
c.execute(query_1.format(company, start_date, end_date))
end_time = time.time()
test1_idx = end_time - start_time

# Printing results
print("Getting all sales to company " + company + " between time " + start_date + " and "
      + end_date + " (" + str(len(c.fetchall())) + " rows) in " + str(test1_idx) + " secs")

In [ ]:
# Executing the SQL query
start_time = time.time()
c.execute(query_2.format(rows))
end_time = time.time()
test2_idx = end_time - start_time

# Printing results
print("Getting the latest " + str(rows) + " sales in " + str(test2_idx) + " secs")

The execution time is being reduced by 90% (from 10 to 1 milliseconds). But let's try with the third most used query.

In [ ]:
# Executing the SQL query
start_time = time.time()
c.execute(query_3.format(rows))
end_time = time.time()
test3_idx = end_time - start_time

# Printing results
print("Getting total sales per company per day (" + str(len(c.fetchall())) + " rows) in " 
      + str(test3_idx) + " secs")

The performance seems to be very similar to the previous one (without the index in the _date_ column) and there is no appreciable performance's improvement in this particular case. May be due to almost all the computational cost is located in the sum of the prices per day and not while filtering by date.

To try to minimize the impact of this situation could be positive to add another index to the _price_ column in the **transactions** table.

In [ ]:
# Reseting the connection to the database
c.close()
conn = sqlite3.connect('project1.db')
c = conn.cursor()

# Creating index in PRICE column (table TRANSACTIONS)
index = "CREATE INDEX index_price ON transactions (price);"
c.execute(index)

In [ ]:
# Executing the SQL query
start_time = time.time()
c.execute(query_3.format(rows))
end_time = time.time()
test3_extra = end_time - start_time

# Printing results
print("Getting total sales per company per day (" + str(len(c.fetchall())) + " rows) in " 
      + str(test3_extra) + " secs")

As we can see, the time is not being reduced clearly after adding the last index. We can say that the query is already optimized using this technique, but it is a good performance anyway (between 20-40 milliseconds).

Indexes are used to implement fast searches on columns with good results. They need additional space consumption and some insertions and updates could take longer, but it is necessary if the system is going to receive a larger number of search queries compared to the number of insertions and updates.

## 5.3 Conclusion

---

The proposed solution is scalable and can be optimized taking a look to the most used queries. If for some reason other queries become more frequent and it is necessary to add more indexes the process is very easy and quick.

---